# Model Training Tesina

## Access Google Drive

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")  
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [2]:
%cd "/content/gdrive/MyDrive/Tesina/"
!ls

/content/gdrive/MyDrive/Tesina
data  model_weights34.pth  model_weights.pth  Tesina.gdoc  Tesina.ipynb


## Setting up libraries

In [3]:
import time
import copy
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder

## Testing installed libraries


In [4]:
x = torch.ones(3,2)
print(x)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


## Setting up model

### Configuration

#### Global variables

In [13]:
PATH = 'data'
BATCH_SIZE=1024
EPOCHS=50
LR=0.001

#### Device configuration

In [6]:
#dev = xm.xla_device()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Current device: {device}')
#print(f'TPU?: {dev}')


Current device: cuda


#### Data preprocessing

In [16]:
# Getting dataset, transforming to tensor and spliting
transform_dataset = T.Compose([
        T.Resize((200,200)),
        T.ToTensor()
    ])

my_dataset = ImageFolder(PATH,transform=transform_dataset)

 # Dataset split
train_set_size = int(len(my_dataset) * 0.8)
test_set_size = len(my_dataset) - train_set_size
train_set, val_set, test_set = random_split(my_dataset, [0.8,0.1,0.1])

train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=True)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

dataloaders = {
        'train':DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True),
        'val':DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=True)
        }

dataset_sizes = {
        'train':len(train_set),
        'val':len(val_set)
        } 



#### Testing Data set up

In [17]:
print('Train data set:', len(train_set))
print(f'Validation set: {len(val_set)}')
print('Test data set:', len(test_set))


Train data set: 8211
Validation set: 1026
Test data set: 1026


### Model Training definition

In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                #print(f'Iteration: [{i+1}/{n_total_steps}]')

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if epoch_acc >0.85:
              break

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



## Transfer Learning

## Model Training

### ResNet 18

In [15]:
# importing resnet for TL
model = models.resnet18()
#freezing parameters
for param in model.parameters():
    param.requires_grad = False


num_features = model.fc.in_features
model.fc = nn.Linear(num_features,2)
#model.fc = model.fc.to(dev)
# Send to device
model = model.to(device)
#train_set, val_set, test_set = train_set.to(dev), val_set.to(dev), test_set.to(dev)

optimizer = torch.optim.SGD(model.fc.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
#criterion = criterion.to(dev)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [28]:
# training model
train_model(model, criterion, optimizer, exp_lr_scheduler, EPOCHS)

# Download Model
torch.save(model.state_dict(), 'model_weights.pth')

train Loss: 0.5615 Acc: 0.7041
val Loss: 0.5533 Acc: 0.7047

Epoch 17/19
----------
train Loss: 0.5629 Acc: 0.7003
val Loss: 0.5516 Acc: 0.7086

Epoch 18/19
----------
train Loss: 0.5616 Acc: 0.7033
val Loss: 0.5517 Acc: 0.7125

Epoch 19/19
----------
train Loss: 0.5613 Acc: 0.7033
val Loss: 0.5525 Acc: 0.7105

Training complete in 128m 4s
Best val Acc: 0.719298


### ResNet 34

In [19]:
model = models.resnet34()
#freezing parameters
for param in model.parameters():
    param.requires_grad = False


num_features = model.fc.in_features
model.fc = nn.Linear(num_features,2)
#model.fc = model.fc.to(dev)
# Send to device
model = model.to(device)

optimizer = torch.optim.SGD(model.fc.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
#criterion = criterion.to(dev)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Train ResNet34
train_model(model, criterion, optimizer, exp_lr_scheduler, 1)

# Download Model
torch.save(model.state_dict(), 'model_weights34.pth')


Epoch 0/0
----------
train Loss: 0.6900 Acc: 0.5655


### ResNet 50

In [15]:
model50 = models.resnet50()
#freezing parameters
for param in model50.parameters():
    param.requires_grad = False


num_features = model50.fc.in_features
model50.fc = nn.Linear(num_features,2)
#model.fc = model.fc.to(dev)
# Send to device

model50 = model50.to(device)

optimizer = torch.optim.SGD(model50.fc.parameters(), lr=LR)
optimizerAdam = torch.optim.Adam(model50.fc.parameters(),lr=LR)
criterion = nn.CrossEntropyLoss()
#criterion = criterion.to(dev)
exp_lr_scheduler = lr_scheduler.StepLR(optimizerAdam, step_size=7, gamma=0.1)

In [ ]:
# Train ResNet50
train_model(model50, criterion, optimizerAdam, exp_lr_scheduler, 1)

# Download Model
torch.save(model50.state_dict(), 'model_weights50.pth')
